In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext  
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils   

In [2]:
batch_interval = 1
window_length = 15 * batch_interval
frequency = 6 * batch_interval

In [3]:
sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, batch_interval)

In [4]:
directKafkaStream = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092'})

In [5]:
lines = directKafkaStream.map(lambda x: x[1])

In [6]:
import re

In [7]:
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0

In [8]:
def tuple_sum(a, b):
    return [a[0]+b[0], a[1]+b[1]]

In [9]:
def tuple_inv(a, b):
    return [a[0]-b[0], a[1]-b[1]]

In [10]:
# ctr = lines.flatMap(lambda line: [line.split("\t")[-1]]) \
#     .map(lambda click: ("ctr", [mk_int(click), 1])) \
#     .reduceByKey(lambda a, b: tuple_sum(a, b)) \
#     .flatMap(lambda row: ("ctr = ", 100.0 * row[1][0] / row[1][1]))

In [11]:
# ctr = lines.flatMap(lambda line: [line.split("\t")[-1]]) \
#     .map(lambda click: ("ctr", [mk_int(click), 1])) \
#     .reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), lambda a, b: tuple_inv(a, b), window_length, frequency) \
#     .flatMap(lambda row: ("ctr = ", 100.0 * row[1][0] / row[1][1]))

In [12]:
ctr = lines.flatMap(lambda line: [line.split("\t")[-1]]) \
    .map(lambda click: ("ctr", [mk_int(click), 1])) \
    .reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), window_length, frequency) 

In [13]:
ctr.pprint()

In [14]:
ssc.start()  
ssc.awaitTermination()




Py4JJavaError: An error occurred while calling o28.start.
: java.lang.IllegalArgumentException: requirement failed: The checkpoint directory has not been set. Please set it by StreamingContext.checkpoint().
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.streaming.dstream.DStream.validateAtStart(DStream.scala:242)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$validateAtStart$8.apply(DStream.scala:275)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$validateAtStart$8.apply(DStream.scala:275)
	at scala.collection.immutable.List.foreach(List.scala:381)
	at org.apache.spark.streaming.dstream.DStream.validateAtStart(DStream.scala:275)
	at org.apache.spark.streaming.DStreamGraph$$anonfun$start$4.apply(DStreamGraph.scala:48)
	at org.apache.spark.streaming.DStreamGraph$$anonfun$start$4.apply(DStreamGraph.scala:48)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.sca

In [14]:
ssc.stop()
sc.stop()